<a href="https://colab.research.google.com/github/gmm/RDKit-on-Colab/blob/main/RDKit_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Brief Introduction to Chemistry

## Molecules, Atoms, and Bonds

All life is made of [molecules](https://en.wikipedia.org/wiki/Molecule), which in turn, are made of [atoms](https://en.wikipedia.org/wiki/Atom) and [bonds](https://en.wikipedia.org/wiki/Chemical_bond). The realm of molecules is incredibly small: too small to see with the naked eye. A water molecule is about 0.27 nm, or 2.7 Ångstrom (1 Å = 10<sup>-10</sup> m), and the simplest possible molecule, dihydrogen, or H<sub>2</sub>, is only 0.74 Å.

Atoms have physical and chemical properties that are determined by their atomic and electronic structure: how many protons in the nucleus (determines the element), how many neutrons in the nucleus (determines the isotope) and how many electrons (determines the charge on the atom).

## The Periodic Table

The elements can be arranged periodically according to their number of protons ("atomic number") and the number of electrons in their outermost shells when they are neutral. These periodic patterns of atomic structure reveal similarities in physicochemical properties, and are exemplified in the [periodic table](https://en.wikipedia.org/wiki/Periodic_table):

![The Periodic Table](https://upload.wikimedia.org/wikipedia/commons/c/c1/Periodic_table_%2832-col%2C_enwiki%29%2C_black_and_white.png "The Periodic Table")
Image Credit: Anonymous, <a href="https://commons.wikimedia.org/wiki/File:Periodic_table_(32-col,_enwiki),_black_and_white.png">Periodic table (32-col, enwiki), black and white</a>, <a href="https://creativecommons.org/licenses/by-sa/4.0/legalcode" rel="license">CC BY-SA 4.0</a>

This video version of the periodic table is particularly fun: [Periodic Videos](http://www.periodicvideos.com)…

## Electrons and Valency

One of the consequences of the quantum mechanical nature of atoms is that their electrons reside in discrete energy levels, sometimes referred to as "[shells](https://en.wikipedia.org/wiki/Electron_shell)". Pre-quantum mechanical ideas visualized these electrons as "orbitting" the nucleus, like planets around a star--but this model is [inaccurate](https://en.wikipedia.org/wiki/Uncertainty_principle). It's better to think of electrons as probability density clouds, which in atoms are known as "[atomic orbitals](https://en.wikipedia.org/wiki/Atomic_orbital)":

<p style="text-align: center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/c/c4/Atomic-orbital-clouds_spdf_m0.png" alt="Atomic Orbitals" width="400" style="display=block; margin:auto"/>
    Image Credit: <a href="https://commons.wikimedia.org/wiki/User:Geek3">Geek3</a>, <a href="https://commons.wikimedia.org/wiki/File:Atomic-orbital-clouds_spdf_m0.png">Atomic-orbital-clouds spdf m0</a>, <a href="https://creativecommons.org/licenses/by-sa/4.0/legalcode" rel="license">CC BY-SA 4.0</a>
</p>

Each of these shells has a maximum capacity to hold electrons, and they fill up with electrons in order from lowest energy to highest. The number of protons in the nucleus, and the number of electrons in the highest energy level--or "outermost shell"--determine the physical and chemcial properties of the element, and how many atoms it can bond to at once, a.k.a. its [valency](https://en.wikipedia.org/wiki/Valence_(chemistry)). You [can use valency to figure out the formula](https://www.bbc.co.uk/bitesize/guides/zqrxsbk/revision/2) of a compound.

## Organic Molecules

[Organic molecules](https://en.wikipedia.org/wiki/Organic_compound) are made up of carbon (C) and hydrogen (H), usually have nitrogen (N) and/or oxygen (O); and can have other elements like phosphorus (P) and (S). Small molecule drugs can have other atoms, like the halogen atoms: fluorine (F), chlorine (Cl), Bromine (Br), and/or iodine (I); and other elements such as boron (B).

## Bonds

Pairs of adjacent atoms in molecules are held together by strong covalent bonds, which are formed when the atom pairs share their outermost electrons. There are different types of covalent bonds: single, double, and triple bonds, in which two, four, and six electrons respectively are shared between the bonded atoms. The strongest bonds are triple, then double, then single; and the stronger the bond, the shorter they tend to be.

The concept of "saturation" relates to how much hydrogen is attached to the carbon atoms and whether the carbons are singly, doubly or triply bonded. Hydrocarbons are compounds made up of hydrogen and carbon. A fully saturated hydrocarbon compound has the highest hydrogen to carbon ratio, while an unsaturated compound has the least hydrogen. For example, ethane has the chemical formula C<sub>2</sub>H<sub>6</sub> and the two carbon atoms are connected by a single bond. Ethene also has two carbon atoms, but it has a double bond between the carbons, and the formula C<sub>2</sub>H<sub>4</sub>. Ethane is saturated, while ethene is unsaturated.

There are also non-covalent bonds, which are weaker than covalent bonds, and form between parts of the same molecule, or between different molecules.

## Hydrogen Bonds

[Hydrogen bonds](https://en.wikipedia.org/wiki/Hydrogen_bond) are weak, directional interactions that arise as a result of differences in [electronegativity](https://en.wikipedia.org/wiki/Electronegativity) between hydrogen (less electronegative) and atoms like oxygen and nitrogen (more electronegative). The difference in electronegativity between two atoms in a bond causes the electrons in the bond(s) between the atoms to be distributed unequally, so one atom becomes slightly positively charged, and the other slightly negative. A hydrogen bond involves a hydrogen bond donor such as an N-H group, and a hydrogen bond acceptor such as an oxygen atom, and is often represented by dotted lines: N-H•••O. H-bonds while weak are highly directional: the angle from the H-bonding donor atom, via the H to the H-bonding acceptor tends to be ~180°. This leads to emergent local structure btween molecules, such as the tetrahedral lattice in water ice; or (secondary structure in proteins)[https://en.wikipedia.org/wiki/Protein_secondary_structure] formed by adjacent backbone amide groups.

## Rings

Molecules are examples of (graphs)[https://en.wikipedia.org/wiki/Graph_theory] where the nodes are atoms and the edges are bonds. Just like graphs, molecules can be linear, branched, or cyclic, where atoms are arranged in rings. A saturated hydrocarbon ring consisting of six carbon atoms connected by six single bonds and is saturated: this molecule is called "cyclohexane", and is known as an "aliphatic ring"; it has the chemical formula C<sub>6</sub>H<sub>12</sub> But is also possible to have a unsaturated hydrocarbon ring of six carbons with alternating single and double bonds: this molecule is called "benzene", and is known as an "aromatic ring"; it has the chemical formula C<sub>6</sub>H<sub>6</sub>.

## Chemical Space

With the combinatorics of elements and their valencies, different bond types, and whether a molecule is linear, branched or cyclic, it's easy to imagine there are lots of possible small molecules. [Bohacek *et al.*](https://onlinelibrary.wiley.com/doi/abs/10.1002/%28SICI%291098-1128%28199601%2916%3A1%3C3%3A%3AAID-MED1%3E3.0.CO%3B2-6?sid=nlm%3Apubmed) claimed that just restricting ourselves to molecules with up to 30 atoms chosen from C, N, O, S, and H atoms, the "true number [of molecules] will be well in excess of 10<sup>60</sup>" [Bohacek *et al.*, 1996, *Med. Res. Rev.*, __16__: 3-50).

## Functional Groups

Often when describing parts of molecules that can vary, chemists use the symbol "R" to describe a group or sub-graph that is generic. When there are several points of variation, a subscripted number is used, such as "R<sub>1</sub>" or "R<sub>2</sub>".

Common 'sub-graphs' in molecules are referred to by chemists as "[functional groups](https://en.wikipedia.org/wiki/Functional_group)". Functional groups have unique physicochemical properties, such as being neutral, negatively or positively charged; or being hydrophobic or hydrophilic. Examples include hydroxyl groups, R<sub>1</sub>(OH); carbonyl groups, R<sub>1</sub>(C=O)R<sub>2</sub>; carboxylic acid groups, R<sub>1</sub>(O=C-OH); and ammonium groups, R<sub>1</sub>(NH<sub>3</sub>). 

## Molecular Properties

Molecules have [chemical properties](https://en.wikipedia.org/wiki/Chemical_property) and [physical properties](https://en.wikipedia.org/wiki/Physical_property) that emerge from their chemical structure: whether they are solid, liquid or gaseous; their melting point if solid, or boiling point if liquid. They also have properties such as molecular weight (the sum of the atomic masses of all the atoms in the molecule); and counts, such as the number of non-hydrogen 'heavy' atoms, or the number of hydrogren bond donors. Cheminformatics software allows us to [calculate many such molecular descriptors](https://www.rdkit.org/docs/GettingStartedInPython.html#descriptor-calculation).


# Cheminformatics

The discipline of cheminformatics is centred on the algorithmic manipulation of chemical information. It allows us to store representations of (ususally small organic) molecules in a computer, and run algorithms on them to compute molecular properties, compare molecules, or search massive databases of molecules. In combination with machine learning, cheminformatics allows us to expand our models to molecular property prediction and even *de novo* design. The open source cheminformatics API, [RDKit](http://www.rdkit.org), provides a wealth of such functionality, that can be accessed from e`ither C++ or [Python](https://www.rdkit.org/docs/GettingStartedInPython.htmlhttps://www.rdkit.org/docs/GettingStartedInPython.html).

## Python

[Python](https://www.python.org) is a powerful general purpose, interpreted language that acts a computational glue for a wide variety of mathematical and scientific algorithms, including cheminformatics, bioinformatics, and biomolecular simulation. It has become extremely popular in data science. [iPython](https://ipython.org) provides a powerful interactive Python environment that also works well with the host operating system using so called [magic commands](https://ipython.readthedocs.io/en/stable/interactive/python-ipython-diff.html), which unlike Python begin with a "%" symbol. For example, "%cd" will change the directory that Python is currently running in. iPython also supports interactive data visualization and the use of GUI (Graphical User Interface) toolkits.

The Python language has two variants: Python 2 and Python 3; while they are very similar, they are not backwards-compatible. This notebook has been written using Python 3 syntax.

## Jupyter Notebooks on Google Colab

Google provides a way to run Python from a web browser. This is done using [Jupyter](https://jupyter.org), which gives us a way to store code and document it in documents called notebooks, with the file extension ``.ipynb``. These notebooks have "cells" in which you can type Python code, or [Markdown](https://daringfireball.net/projects/markdown/)-formatted text. These Jupyter iPython notebooks run on Google servers remotely using "Google Colab". This iPython Notebook has been developed to run on Google Colab.

Google Colab connects to remote virtual machines known as Google Compute Engine (GCE), and offers different types of runtimes which can be changed from the Runtime menu. Google Colab is a free service, but it is possible to pay for higher tier service that includes access to more memory, GPUs (Graphics Processing Units) and TPUs (Tensor Processing Units).

## Software Package Managers

Software can have many dependencies so we use package managers to take care of tracking these details. In Python, we can install extra functionality using the software package manager "[Conda](https://docs.conda.io/en/latest/)". Conda automatically includes a large number of Python packages like [NumPy](https://numpy.org) and [SciPy](https://scipy.org), but it doesn't include RDKit.

We start by installing Conda and Mamba, using them to install the additional Python packages we need -- most importantly, RDKit.

## 1. Install the Python Package Manager

First we need to install conda before we can install RDKit. We use CondaColab, which was developed by [Jaime Rodríguez-Guerra](https://github.com/jaimergp), a solution that is discussed here:

* https://inside-machinelearning.com/en/how-to-install-use-conda-on-google-colab/.

This needs to be run at the start of the notebook, because it restarts the kernel.

CondaColab uses [Mamba](https://mamba.readthedocs.io/en/latest/index.html), a "fast, robust, and cross-platform package manager, ... a Python-based CLI conceived as a drop-in replacement for conda, offering higher speed and more reliable environment solutions." Mamba's [core is implemented in C++ and allows parallel downloading of repository data and package files using multi-threading](https://github.com/mamba-org/mamba).

By default, CondaColab installs Mambaforge,  but `condacolab.install_anaconda()` [will install the Anaconda 2020.02 distribution, the last version that was built for Python 3.7](https://github.com/jaimergp/condacolab) (which is needed for Google Colab as of July 2021).

Don't panic when you see "Your session crashed for an unknown reason." pop up: the kernel restarts automatically so it has access to the newly installed software, but Colab picks this up as an error. You want this to happen, so you can use the new `conda` environment.

In [1]:
# Running this cell is only necessary when running this Notebook on Google Colab

!time pip install -q condacolab  # ~5s to run
import condacolab
condacolab.install()  # for ML, use `condacolab.install_anaconda()`

# When finished, the kernel will restart automatically with the new conda
# functionality, and show message:
# "Your session crashed for an unknown reason."
# --Click the "X" to dismiss it.


real	0m4.394s
user	0m3.032s
sys	0m0.324s
⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:46
🔁 Restarting kernel...


Check the versions of conda and mamba: they should be 4.9.2 and 0.8.0 or later, respectively.

In [1]:
!which conda
!conda --version
!which mamba
!mamba --version

/usr/local/bin/conda
conda 4.9.2
/usr/local/bin/mamba
mamba 0.8.0
conda 4.9.2


# 2. Update Conda

In tests, `Mamba` seems to be slightly faster than `Conda` to update all the packages:

In [ ]:
#!time conda update -y -n base conda

In [2]:
!time mamba update -y -n base conda  # ~16s to install


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.8.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['conda']

conda-forge/noarch       [] (00m:00s) 
conda-forge/noarch       [] (00m:00s) 15  B / ?? (

In [3]:
!conda --version
!mamba --version

conda 4.10.3
mamba 0.8.0
conda 4.10.3


# 3. Install RDKit

Use mamba to install RDKit.

In [4]:
# Install the latest version of RDKit using mamba
#
#  -- Hint: don't create a separate environment, otherwise you must activate
#           it before Google's Jupyter starts

!time mamba install -y -c rdkit rdkit  # ~41s to install

Streaming output truncated to the last 5000 lines.
pytz                     [] (00m:00s) 
Finished lcms2                                (00m:01s)             443 KB      3 MB/s
libstdcxx-ng             [] (00m:01s) Waiting...
boost-cpp                [] (00m:01s) Validating...
libcblas                 [] (00m:01s) Decompressing...
liblapack                [] (00m:01s) Decompressing...
libuuid                  [] (00m:01s) Waiting...
libpng                   [] (00m:01s) Waiting...
libgfortran5             [] (00m:01s) Waiting...
libxcb                   [] (00m:01s) Waiting...
pcre                     [] (00m:01s) Waiting...
pycairo                  [] (00m:01s) Waiting...
openjpeg                 [] (00m:01s) Waiting...
pixman                   [] (00m:01s) Waiting...
python-dateutil          [] (00m:01s) Waiting...
typing_extensions        [] (00m:01s) Waiting...
numpy                    [] (00m:01s) Waiting...
xorg-libice              [] (00m:01s) Waiting...
rdkit                   

# 4. Install Py3Dmol
Py3Dmol lets us view and interact with 3D molecules directly in iPython and Jupyter Notebooks.

In [5]:
!time mamba install -y -c conda-forge py3dmol  # ~1 min to install

Streaming output truncated to the last 5000 lines.
libogg                   [] (00m:02s) Waiting...
mistune                  [] (00m:02s) Waiting...
nspr                     [] (00m:01s) Waiting...
send2trash               [] (00m:01s) Waiting...
prometheus_client        [] (00m:01s) Waiting...
backcall                 [] (00m:01s) Waiting...
py3dmol                  [] (00m:01s) Waiting...
jupyterlab_widgets       [] (00m:01s) Waiting...
gst-plugins-base         [] (00m:01s) Waiting...
nbconvert                [] (00m:01s) Waiting...
wcwidth                  [] (00m:01s) Waiting...
pyqt5-sip                [] (00m:01s) Waiting...
jedi                     [] (00m:01s) Waiting...
pandocfilters            [] (00m:01s) Waiting...
dbus                     [] (00m:01s) Waiting...
pyzmq                    [] (00m:00s) Waiting...
libvorbis                [] (00m:00s) Waiting...
nbclient                 [] (00m:04s) Decompressing...
nbformat                 [] (00m:04s) Waiting...
libclang    

In [6]:
%pwd

'/content'

In [7]:
%ls

condacolab_install.log  sample_data/


## 4.1 Install MAP4 Fingerprints

From "One molecular fingerprint to rule them all: drugs, biomolecules, and  the metabolome" by Reymond /et al./ https://jcheminf.biomedcentral.com/articles/10.1186/s13321-020-00445-4

In [ ]:
# MAP4 depends on TMAP:
!time mamba install -y -c tmap tmap

In [ ]:
##!wget https://github.com/reymond-group/map4/raw/master/environment.yml
##!mamba env update -n base --file environment.yml
!time pip install git+https://github.com/reymond-group/map4@v1.0

In [ ]:
from rdkit import Chem
import tmap as tm
from map4 import MAP4Calculator

smiles_1 = 'c1ccccc1'
m1 = Chem.MolFromSmiles(smiles_1)

smiles_2 = 'c1cccc(N)c1'
m2 = Chem.MolFromSmiles(smiles_2)

def get_map4_similarity_of_mols(m1, m2, dim=1024):
  """Calculate the minhashed distance between the MAP4 fingerprints of 
two molecules.

See: https://github.com/reymond-group/map4/blob/master/test.py

>>> print(get_map4_similarity_of_mols(m1,m2)
0.7861328125
"""
  MAP4 = MAP4Calculator(dimensions=dim)
  ENC = tm.Minhash(dim)
  map4_m1 = MAP4.calculate(m1)
  map4_m2 = MAP4.calculate(m2)

  # or use parallelized version:
  fps = MAP4.calculate_many([m1, m2])

  return ENC.get_distance(map4_m1, map4_m2)
  #return ENC.get_distance(fps[0], fps[1])

print(get_map4_similarity_of_mols(m1,m2))

# 5. Import RDKit

There are lots of useful websites explaining how to use RDKit:

*   [Getting Started with the RDKit in Python](https://rdkit.readthedocs.io/en/latest/GettingStartedInPython.html)
* [Greg Landrum's RDKit Blog](https://greglandrum.github.io/rdkit-blog)
** [Generating 3D conformers of molecules](https://greglandrum.github.io/rdkit-blog/conformers/exploration/2021/02/22/etkdg-and-distance-constraints.html)
*   https://xinhaoli74.github.io/blog/
**    https://xinhaoli74.github.io/blog/rdkit/2021/01/06/rdkit.html



In [8]:
import rdkit
print(rdkit.__version__)

2021.09.2


In [9]:
from rdkit import Chem

In [10]:
from rdkit.Chem.Draw import IPythonConsole # to draw inline in iPython
from rdkit.Chem import Draw # to draw molecules

In [11]:
# Change the default to show molecules in interactive 3D
IPythonConsole.ipython_3d = True

# later on, get a conformer from an RDKit molecule, and use
# IPythonConsole.drawMol3D(m, confID=cids[0])

# See: https://greglandrum.github.io/rdkit-blog/conformers/exploration/2021/02/22/etkdg-and-distance-constraints.html

In [12]:
from rdkit.Chem import PandasTools # we use this to read in SDFs into pandas dataframes

In [13]:
from IPython.display import SVG # to use vectors, not bitmaps, for cleaner lines
from rdkit.Chem import rdDepictor  # to generate 2D depictions of molecules
from rdkit.Chem.Draw import rdMolDraw2D # to draw 2D molecules using vectors

In [14]:
import pandas as pd # we need this to make copies of pandas columns of molecules

In [15]:
from rdkit.Chem import AllChem  # we need this to compute 2D depictions
from copy import deepcopy  # we need deep copies of the molecule to avoid losing 3D coordinates

In [16]:
import py3Dmol # for inline 3D interactive views of molecules

# 6. Let's Draw Some Molecules!

We can create molecules by starting from a string that describes the elements, bonds (and their connectivity), stereochemistry, and charge of the molecule, using SMILES ([Simplified Molecular Input Line Entry System](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system)). Branches are written inside parentheses, while atoms in rings are closed by writing numeric digits after the atoms:

<img src="https://upload.wikimedia.org/wikipedia/commons/0/00/SMILES.png" width="400"/>
Image Credit: Original by <a href="https://commons.wikimedia.org/wiki/User:Fdardel">Fdardel</a>, slight edit by <a href="https://commons.wikimedia.org/wiki/User:DMacks">DMacks</a>, <a href="https://commons.wikimedia.org/wiki/File:SMILES.png">SMILES</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0/legalcode" rel="license">CC BY-SA 3.0</a>

* [SMILES Examples](https://www.daylight.com/dayhtml_tutorials/languages/smiles/smiles_examples.html)
* [SMILES Tutorial](https://daylight.com/dayhtml_tutorials/languages/smiles/index.html)


In [ ]:
mol = Chem.MolFromSmiles('c1ccccc1C(=O)N')
mol

In [ ]:
def show_atom_numbers(mol, label):
  """See https://stackoverflow.com/questions/53321453/rdkit-how-to-show-moleculars-atoms-number?answertab=active#tab-top"""
  for atom in mol.GetAtoms():
    atom.SetProp(label, str(atom.GetIdx()+1))
  return mol

In [ ]:
mol = Chem.MolFromSmiles('c1ccccc1C(=O)N')
show_atom_numbers(mol, 'atomLabel')

In [ ]:
mol = Chem.MolFromSmiles('c1ccccc1C(=O)N')
show_atom_numbers(mol, 'molAtomMapNumber')

In [ ]:
mol = Chem.MolFromSmiles('c1ccccc1C(=O)N')
show_atom_numbers(mol, 'atomNote')

# 7. Draw Sharper Molecules Using Vectors, not Bitmaps

This code uses SVG (Scalar Vector Graphics) to depict molecules using vector graphics.

See: https://leedavies.dev/index.php/2018/10/06/rdkit-in-jupyter-notebooks/

In [ ]:
# Create mol object from smiles string
mol = Chem.MolFromSmiles('c1cccnc1C(=O)N')

molSize=(450,150)
mc = Chem.Mol(mol.ToBinary())
if not mc.GetNumConformers():
  #Compute 2D coordinates
  rdDepictor.Compute2DCoords(mc)

# init the drawer with the size
drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])

#draw the molcule
drawer.DrawMolecule(mc)
drawer.FinishDrawing()

# get the SVG string
svg = drawer.GetDrawingText()

# fix the svg string and display it
display(SVG(svg.replace('svg:','')))

# 8. Get Drugs from DrugBank

Free access to DrugBank is permitted for students, academics and non-profits; but you need to apply for access, then use your username and password to login or `wget` files (using `--user` and `--password`).

See: https://go.drugbank.com/releases/latest#structures

In [ ]:
### WARNING! NEVER EMBED USERNAME AND PASSWORD!!! ###
### ESPECIALLY BEFORE SAVING OR COMMITTING CHANGES!!! ###

import getpass
user = getpass.getpass("DrugBank username: ")
pwd = getpass.getpass("DrugBank password: ")

#!wget --user $user --password $pwd https://go.drugbank.com/releases/5-1-8/downloads/all-structures
!wget --user $user --password $pwd https://go.drugbank.com/releases/5-1-8/downloads/all-3d-structures
#!wget --user $user --password $pwd https://go.drugbank.com/releases/5-1-8/downloads/all-structure-links

In [ ]:
%ls

In [ ]:
!unzip all-3d-structures 
#!unzip all-structure-links 
#!unzip all-structures

In [ ]:
%ls

In [ ]:
!mv 3D\ structures.sdf all-drugbank-3D.sdf

In [ ]:
%ls

# 9. Draw Molecules in a Grid

In [ ]:
smiles = [ 'N#CC(OC1OC(COC2OC(CO)C(O)C(O)C2O)C(O)C(O)C1O)c1ccccc1',
           'c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21',
           'C=C(C)C1Cc2c(ccc3c2OC2COc4cc(OC)c(OC)cc4C2C3=O)O1',
           'ClC(Cl)=C(c1ccc(Cl)cc1)c1ccc(Cl)cc1']

mols = [Chem.MolFromSmiles(smi) for smi in smiles]

In [ ]:
Draw.MolsToGridImage(mols, molsPerRow=2, subImgSize=(300, 300))

# 10. Get 3D Structures of Drugs from DrugBank

This will include everything: approved, withdrawn, illicit drugs and nutraceuticals, etc.

In [ ]:
# Read in the 3D coordinates of all the drugs in DrugBank
# See Susan Leung's Blopig post: https://www.blopig.com/blog/2017/02/using-rdkit-to-load-ligand-sdfs-into-pandas-dataframes/
filename = 'all-drugbank-3D.sdf'
drugbank = PandasTools.LoadSDF(filename)

In [ ]:
drugbank.info()

In [ ]:
drugbank[:3]

In [ ]:
drugbank[['GENERIC_NAME', 'MOLECULAR_WEIGHT', 'ROMol']][:3]

## 11. Make 2D Copies of the Molecules

If we don't make deep copies of each molecule, Compute2DCoords will overwrite the original 3D coordinates of the atoms.

In [ ]:
copy_of_mols = pd.Series( deepcopy(drugbank['ROMol'].to_dict()) ) 
# See: https://stackoverflow.com/questions/52708341/make-a-truly-deep-copy-of-a-pandas-series
for m in copy_of_mols:
  _ = AllChem.Compute2DCoords(m) # only updates coords of m
drugbank_2D = [m for m in copy_of_mols]

drugbank['ROMol_2D'] = drugbank_2D

In [ ]:
drugbank[['GENERIC_NAME', 'MOLECULAR_WEIGHT', 'ROMol_2D']][:3]

In [ ]:
drug = drugbank['ROMol'][2]

In [ ]:
drug

In [ ]:
IPythonConsole.drawMol3D(drug)

In [ ]:
view = py3Dmol.view(query='pdb:1hvr')
view.setStyle({'cartoon':{'color':'spectrum'}})
view

In [ ]:
IPythonConsole.ipython_3d = False

In [ ]:
drug

In [ ]:
IPythonConsole.ipython_3d = True

In [ ]:
drug